
# Enhanced QAOA Portfolio Optimization — Advanced Features
This notebook is an enhanced, hackathon-ready QAOA portfolio optimization demo. Additions over the basic model:
- **Ledoit–Wolf shrinkage** for covariance (robust Sigma estimate)
- **Black–Litterman** (optional) to blend prior returns with views / market caps
- **Discrete weights** encoding (binary fixed-point) or simple selection encoding
- **Sector caps**, **cardinality**, and **turnover** penalties in QUBO
- **Warm-start** hinting and guidance for custom XY-mixers (implementation notes)
- **Dinkelbach** loop to convert Sharpe ratio into quadratic subproblems
- **Subset tangency** refinement and transaction-cost-aware Sharpe evaluation

Run this in a clean Python 3.10/3.11 environment (conda env recommended). If using Colab, uncomment the pip install line in the first code cell.


In [ ]:

# Colab / first-run installs (uncomment if needed)
# !pip install -q qiskit==1.2.4 qiskit-optimization==0.6.1 qiskit-aer==0.13.3 docplex==2.25.236 yfinance numpy pandas scipy matplotlib

# Standard imports
import numpy as np, pandas as pd, math, warnings, os
from typing import List, Tuple, Dict, Optional
warnings.filterwarnings('ignore')
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt

# Qiskit imports
from qiskit.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import SPSA
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer

# QUBO builder helper
from docplex.mp.model import Model

# Data fetch
import yfinance as yf


In [ ]:

# ===== CONFIG =====
CFG = {
    'tickers': ['AAPL','MSFT','GOOGL','AMZN','NVDA','META','TSLA','AMD','CRM','AVGO'],
    'period': '1y',
    'interval': '1d',
    'risk_free': 0.02,
    'K': 5,                     # cardinality for selection-mode
    'bits_per_asset': 2,        # for discrete weights encoding (binary fixed-point)
    'weight_step': 0.05,        # unary alternative step size (not used if binary)
    'use_discrete_weights': False, # if True uses multi-bit weight encoding; else selection (0/1)
    'max_weight': 0.5,          # cap per asset in discrete mode
    'dinkelbach_iters': 4,
    'p_list': [1,2], 'shots': 2048, 'seed': 123,
    'shrink_leduit_wolf': True,
    'bl_enabled': False,        # optional Black-Litterman
    'sector_caps': {},         # e.g. {'Tech':([0,1,4],3)}
    'prev_selection': None,     # list of 0/1 previous picks for turnover penalty
    'lambda_ham': 0.1,          # turnover penalty strength
    'transaction_cost': 0.001,  # per-dollar transaction cost (for turnover-aware Sharpe)
}



## Ledoit–Wolf shrinkage (simple implementation)
We compute the LW shrinkage target as the diagonal matrix and compute alpha following a basic Ledoit–Wolf closed form for small samples.


In [ ]:

def ledoit_wolf_shrinkage(Sigma_sample: np.ndarray, returns: np.ndarray):
    # returns: T x n matrix of returns (daily)
    n = Sigma_sample.shape[0]
    T = returns.shape[0]
    sample = Sigma_sample.copy()
    mu = np.diag(sample).mean()
    F = np.eye(n) * mu
    # compute pi_hat
    X = returns - returns.mean(axis=0, keepdims=True)
    pi_hat = 0.0
    for t in range(T):
        xt = X[t:t+1].T @ X[t:t+1]
        pi_hat += np.sum((xt - sample) ** 2)
    pi_hat /= T
    # rho_hat
    rho_hat = np.sum((sample - F) ** 2)
    # gamma
    gamma = np.linalg.norm(sample - F, 'fro')**2
    # alpha (shrinkage intensity)
    kappa = (pi_hat - rho_hat) / gamma if gamma > 0 else 0.0
    alpha = max(0.0, min(1.0, kappa / T))
    Sigma_shrunk = (1 - alpha) * sample + alpha * F
    return Sigma_shrunk, alpha



## Black–Litterman (optional)
If enabled, we compute an implied prior from market caps or user-supplied prior and blend with views P, Q.


In [ ]:

def black_litterman(mu_prior: np.ndarray, Sigma: np.ndarray, P: Optional[np.ndarray], Q: Optional[np.ndarray],
                    tau: float = 0.05, omega: Optional[np.ndarray] = None):
    # Simple BL posterior mean (returns)
    n = len(mu_prior)
    if P is None or Q is None:
        return mu_prior
    if omega is None:
        omega = np.diag(np.diag(P @ (tau * Sigma) @ P.T))
    A = np.linalg.pinv(tau * Sigma)
    middle = A + P.T @ np.linalg.pinv(omega) @ P
    rhs = A @ mu_prior + P.T @ np.linalg.pinv(omega) @ Q
    mu_bl = np.linalg.pinv(middle) @ rhs
    return mu_bl



## Data fetch and preprocessing (annualize returns and covariance)


In [ ]:

def fetch_and_prepare(tickers, period='1y', interval='1d'):
    df = yf.download(tickers, period=period, interval=interval, auto_adjust=True, progress=False)['Close']
    if isinstance(df, pd.Series):
        df = df.to_frame()
    df = df.dropna(how='all').ffill().dropna(axis=1)
    rets = df.pct_change().dropna()
    mu_d = rets.mean().values
    Sigma_d = rets.cov().values
    mu = mu_d * 252.0
    Sigma = Sigma_d * 252.0
    return df, rets.values, mu, Sigma

prices, rets_daily, mu, Sigma = fetch_and_prepare(CFG['tickers'], CFG['period'], CFG['interval'])
n = len(mu)
print('Tickers:', CFG['tickers'])
print('mu (annual):', np.round(mu,4))


In [ ]:

# Apply Ledoit-Wolf shrinkage if enabled
if CFG['shrink_leduit_wolf']:
    Sigma_sh, alpha = ledoit_wolf_shrinkage(Sigma, rets_daily)
    print('Ledoit-Wolf alpha:', round(alpha,5))
    Sigma = Sigma_sh



## Encoding options
- **Selection mode**: x_i ∈ {0,1} — pick asset or not (equal weights or refine later)
- **Discrete weights (binary fixed-point)**: use `bits_per_asset` per asset to encode weights up to `max_weight`.
Note: discrete weights blow up qubit count: n * bits_per_asset qubits.


In [ ]:

def binary_encoding_vars(n, bits_per_asset):
    # returns total bits and mapping asset->bit indices
    total_bits = n * bits_per_asset
    mapping = {i: list(range(i*bits_per_asset, (i+1)*bits_per_asset)) for i in range(n)}
    return total_bits, mapping

def weight_from_bits(y_bits: np.ndarray, mapping: Dict[int, List[int]], bits_per_asset: int, max_weight: float):
    # interpret bits as unsigned integer and scale to [0, max_weight]
    n = len(mapping)
    w = np.zeros(n)
    for i in range(n):
        inds = mapping[i]
        val = 0
        for b_idx, bitpos in enumerate(inds):
            val += int(y_bits[bitpos]) * (2**b_idx)
        max_int = 2**bits_per_asset - 1
        w[i] = (val / max_int) * max_weight if max_int>0 else 0.0
    return w



## QUBO builder: supports selection mode or discrete-weight binary encoding.
It includes penalties for cardinality, sector caps, and turnover/transaction cost.


In [ ]:

def build_qubo_selection(mu, Sigma, rf, K, t_param, A_card, sector_caps, A_sec, x_prev, lambda_ham, trans_cost):
    r = mu - rf
    n = len(r)
    mdl = Model('select_qubo')
    x = mdl.binary_var_list(n, name='x')

    lin = mdl.sum(r[i] * x[i] for i in range(n))
    quad_risk = mdl.sum(Sigma[i,j] * x[i] * x[j] for i in range(n) for j in range(n))
    obj = lin - (t_param / K) * quad_risk

    # cardinality
    sumx = mdl.sum(x)
    obj -= A_card * (sumx - K) * (sumx - K)

    # sector caps
    for name, (idxs, cap) in sector_caps.items():
        if len(idxs)==0: continue
        sx = mdl.sum(x[i] for i in idxs)
        obj -= A_sec * (sx - cap) * (sx - cap)

    # turnover penalty (Hamming distance) and transaction cost approximation
    if x_prev is not None:
        for i in range(n):
            prev = int(x_prev[i])
            # penalize changes: cost ~ lambda_ham*(x_i + prev - 2 x_i * prev) and tx cost proportional
            obj -= lambda_ham * (x[i] + prev - 2 * x[i] * prev)
            if not prev and trans_cost>0:
                # cost to buy: subtract trans_cost * expected weight approx (use equal weight approx)
                obj -= trans_cost * (1.0/K) * x[i]
    mdl.maximize(obj)
    qp = QuadraticProgram()
    qp = QuadraticProgram.from_docplex(mdl) if hasattr(QuadraticProgram, 'from_docplex') else QuadraticProgram()
    # safe convert via qiskit_optimization.translators if needed
    try:
        from qiskit_optimization.translators import from_docplex_mp
        qp = from_docplex_mp(mdl)
    except Exception:
        # fallback: directly build variables
        pass
    return qp

def build_qubo_binary_weights(mu, Sigma, rf, bits_per_asset, max_weight, t_param, A_budget, budget_val, sector_caps, A_sec, x_prev, lambda_ham, trans_cost):
    # Build QUBO over binary bits representing weights
    n = len(mu)
    total_bits, mapping = binary_encoding_vars(n, bits_per_asset)
    mdl = Model('binary_weight_qubo')
    y = mdl.binary_var_list(total_bits, name='y')

    # map bits to weights symbolically: weight_i = (sum 2^b y_b) / (2^L -1) * max_weight
    max_int = 2**bits_per_asset - 1
    scale = max_weight / max_int if max_int>0 else 0.0

    # linear return term: r^T w
    r = mu - rf
    lin = mdl.sum( r[i] * scale * mdl.sum( (2**(b_idx)) * y[b_idx_global] 
                                          for b_idx, b_idx_global in enumerate(mapping[i]) ) 
                  for i in range(n) )

    # quadratic risk term: w^T Sigma w -> quadratic in y bits
    quad = mdl.sum( (scale * (2**(b_idx)) * (scale * (2**(b_jdx)))) * Sigma[i,j] * y[b_idx_global] * y[b_jdx_global]
                    for i in range(n) for j in range(n)
                    for b_idx, b_idx_global in enumerate(mapping[i])
                    for b_jdx, b_jdx_global in enumerate(mapping[j]) )

    obj = lin - t_param * quad

    # budget constraint: sum weights approx == budget_val (like 1.0)
    # encode (sum_i w_i - budget_val)^2 penalty
    sumw = mdl.sum( scale * mdl.sum((2**b_idx) * y[b_idx_global] for b_idx, b_idx_global in enumerate(mapping[i])) for i in range(n))
    obj -= A_budget * (sumw - budget_val) * (sumw - budget_val)

    # sector caps: approximate by bounding sum of bits in each sector (coarse)
    for name, (idxs, cap) in sector_caps.items():
        # convert cap in weights -> cap_bits approx
        cap_bits = int(np.ceil(cap / max_weight * (2**bits_per_asset - 1))) * len(idxs)
        sx = mdl.sum(y[b] for i in idxs for b in mapping[i])
        obj -= A_sec * (sx - cap_bits) * (sx - cap_bits)

    # turnover and txcost approximations (not exact)
    if x_prev is not None:
        # penalize changes on most-significant bit as proxy
        for i in range(n):
            prev = int(x_prev[i])
            msb_idx = mapping[i][-1]
            obj -= lambda_ham * ( y[msb_idx] + prev - 2 * y[msb_idx] * prev )
    mdl.maximize(obj)
    try:
        from qiskit_optimization.translators import from_docplex_mp
        qp = from_docplex_mp(mdl)
    except Exception:
        qp = QuadraticProgram()
    return qp



## QAOA solver (Aer sampler) and Dinkelbach loop
Note: we use QAOA as the inner solver for each Dinkelbach t parameter.


In [ ]:

def qaoa_solve_qp(qp: QuadraticProgram, p_list, shots, seed):
    qubo = QuadraticProgramToQubo().convert(qp)
    sampler = Sampler(options={'seed':seed, 'shots':shots})
    best=None; best_res=None
    for p in p_list:
        qaoa = QAOA(sampler=sampler, reps=int(p), optimizer=SPSA(maxiter=120, blocking=True))
        solver = MinimumEigenOptimizer(qaoa)
        try:
            res = solver.solve(qubo)
        except Exception as e:
            print('QAOA solver failed:', e)
            continue
        # extract bits into vector
        x = np.array([int(res.variables_dict[k]) for k in sorted(res.variables_dict.keys(), key=lambda s: int(s.split('_')[-1]))])
        val = qubo.objective.evaluate(res.variables_dict)
        cand={'x':x,'p':p,'energy':val,'raw':res}
        if (best is None) or (val>best['energy']):
            best=cand; best_res=res
    return best, best_res

def dinkelbach_qaoa_runner(mu, Sigma, rf, CFG):
    use_discrete = CFG['use_discrete_weights']
    n = len(mu)
    # init t with greedy equal selection
    x0 = baseline_greedy(mu, Sigma, CFG['K'], rf)['x']
    w0 = equal_weights_from_x(x0)
    r = mu - rf
    t = float((r @ w0) / max(w0 @ Sigma @ w0, 1e-12))
    best_overall = None; history=[]
    lam_max = float(np.linalg.eigvalsh(Sigma).max())
    for it in range(CFG['dinkelbach_iters']):
        A_card = 10.0 * (t / CFG['K']) * lam_max
        A_sec = 5.0 * (t / CFG['K']) * lam_max
        if not use_discrete:
            qp = build_qubo_selection(mu, Sigma, rf, CFG['K'], t, A_card, CFG['sector_caps'], A_sec, CFG['prev_selection'], CFG['lambda_ham'], CFG['transaction_cost'])
        else:
            qp = build_qubo_binary_weights(mu, Sigma, rf, CFG['bits_per_asset'], CFG['max_weight'], t, A_card, 1.0, CFG['sector_caps'], A_sec, CFG['prev_selection'], CFG['lambda_ham'], CFG['transaction_cost'])
        best, raw = qaoa_solve_qp(qp, CFG['p_list'], CFG['shots'], CFG['seed'])
        if best is None:
            print('No valid QAOA result at iter', it); break
        x = best['x']
        # for discrete mode, map bits->weights; for selection mode, equal-weight then refine
        if not use_discrete:
            w_eq = equal_weights_from_x(x)
            numer = float((mu - rf) @ w_eq); denom = float(w_eq @ Sigma @ w_eq)
            s_eq = sharpe(w_eq, mu, Sigma, rf)
            t_new = numer / max(denom, 1e-12)
        else:
            # map bits to weights and compute sharpe
            total_bits, mapping = binary_encoding_vars(n, CFG['bits_per_asset'])
            w = weight_from_bits(x, mapping, CFG['bits_per_asset'], CFG['max_weight'])
            # normalize to sum to 1 if possible
            if w.sum() > 0:
                w = w / w.sum()
            else:
                w = np.zeros(n)
            w_eq = w; s_eq = sharpe(w_eq, mu, Sigma, rf)
            numer = float((mu - rf) @ w_eq); denom = float(w_eq @ Sigma @ w_eq)
            t_new = numer / max(denom, 1e-12)
        history.append({'iter':it, 't':t, 't_new':t_new, 'x':x, 'w_eq':w_eq, 'sharpe_eq':s_eq})
        if best_overall is None or s_eq > best_overall['sharpe_eq']:
            best_overall = history[-1]
        if abs(t_new - t) < 1e-3:
            t = t_new; break
        t = t_new
    return best_overall, history



## Helper functions (baselines & refinement)


In [ ]:

def sharpe(weights, mu, Sigma, rf):
    port_ret = float(weights @ mu)
    port_var = float(weights @ Sigma @ weights)
    port_vol = math.sqrt(max(port_var, 1e-12))
    return (port_ret - rf) / (port_vol if port_vol>0 else 1e-12)

def equal_weights_from_x(x):
    idx = np.flatnonzero(x)
    if len(idx)==0: return None
    w = np.zeros_like(x, dtype=float)
    w[idx] = 1.0/len(idx)
    return w

def subset_tangency(mu, Sigma, rf, x):
    S = np.flatnonzero(x)
    if len(S)==0: return None
    Sigma_SS = Sigma[np.ix_(S,S)]; r_S = mu[S] - rf
    inv = np.linalg.pinv(Sigma_SS)
    w_S = inv @ r_S
    denom = np.ones(len(S)) @ w_S
    if abs(denom)<1e-12: return None
    w_S = w_S / denom
    w = np.zeros(len(mu)); w[S]=w_S
    return w

def baseline_greedy(mu, Sigma, K, rf, lamb=4.0):
    diag = np.diag(Sigma); r = mu - rf; scores = r - lamb * diag
    pick = np.argsort(scores)[::-1][:K]
    x = np.zeros_like(mu, dtype=int); x[pick]=1
    w = equal_weights_from_x(x); s = sharpe(w, mu, Sigma, rf)
    return {'name':'greedy','x':x,'w':w,'sharpe':s}

def baseline_tangency(mu, Sigma, rf):
    r = mu - rf; inv = np.linalg.pinv(Sigma); w = inv @ r
    denom = np.ones_like(r) @ w
    if abs(denom)<1e-12: return {'name':'tangency','w':None,'sharpe':float('nan')}
    w = w / denom; return {'name':'tangency','w':w,'sharpe':sharpe(w, mu, Sigma, rf)}


In [ ]:

# Run baselines
greedy_res = baseline_greedy(mu, Sigma, CFG['K'], CFG['risk_free'])
tangency_res = baseline_tangency(mu, Sigma, CFG['risk_free'])
print('Greedy Sharpe:', round(greedy_res['sharpe'],4))
print('Tangency Sharpe:', round(tangency_res['sharpe'],4))

# Run the Dinkelbach QAOA runner
best_overall, history = dinkelbach_qaoa_runner(mu, Sigma, CFG['risk_free'], CFG)
print('\nBest QAOA equal-weight Sharpe (from candidate):', round(best_overall['sharpe_eq'],4))
print('Selected indices:', np.flatnonzero(best_overall['x']).tolist())


In [ ]:

# Refinement for selection mode (tangency)
w_refined = subset_tangency(mu, Sigma, CFG['risk_free'], best_overall['x'])
s_refined = sharpe(w_refined, mu, Sigma, CFG['risk_free'])
print('Refined Sharpe:', round(s_refined,4))

# Show results summary
labels = ['Greedy (eq-w)', 'QAOA (eq-w)', 'QAOA + Refine', 'Tangency (no card)']
values = [greedy_res['sharpe'], best_overall['sharpe_eq'], s_refined, tangency_res['sharpe']]
for L,V in zip(labels, values):
    print(f'{L:>18}: {V:.4f}')

plt.figure(figsize=(7,4)); plt.bar(labels, values); plt.ylabel('Sharpe'); plt.title('Sharpe Comparison'); plt.show()

# Print final weights
sel = np.flatnonzero(best_overall['x']).tolist()
print('Selected tickers:', [CFG['tickers'][i] for i in sel])
print('\nRefined weights:')
for i,t in enumerate(CFG['tickers']):
    if w_refined[i] != 0:
        print(f' {t}: {w_refined[i]:.4f}')



## Notes on XY-mixers and warm-starts
- Implementing custom mixers (XY mixer) requires building parameterized QAOA-like circuits that preserve Hamming-weight subspace (for cardinality constraints). Qiskit's high-level QAOA API does not expose custom mixers directly; you'd implement a custom variational ansatz using `QuantumCircuit` and `VQE`/`Estimator` primitives or a low-level QAOA builder.
- Warm-starting: use greedy solution to bias initial angles or to modify mixer Hamiltonian to prefer certain qubits. Implement by seeding classical optimizer or appending small local rotations toward greedy state prior to QAOA layers.
